In [ ]:
# Call in libraries
import datetime as dt
import time
import logging
import math
import random
from IPython.display import clear_output
from optibook.synchronous_client import Exchange
import pandas as pd
import numpy as np
from cointegration_analysis import estimate_long_run_short_run_relationships, engle_granger_two_step_cointegration_test
import matplotlib.pyplot as plt

# Connect to Exchange
e = Exchange()
a = e.connect()

# Set up logging
logging.getLogger("client").setLevel("ERROR")

# Defining rules 
max_positions = 100
sleep = 0.5

In [ ]:
# Reading in Data
df = pd.read_csv("data.csv")

# Converting DataFrames to log prices
pairs = df[["TOTAL", "UNILEVER"]]

stock_names = list(market_data_log.columns.get_level_values(0).unique())

x = np.log(pairs.iloc[:, 0])
y = np.log(pairs.iloc[:, 1])

In [ ]:
# Return one of the zero pairs randomly
def randomizer(position_zero_pairs):
    pair = random.choice(position_zero_pairs)
    return pair

In [ ]:
# Returns the data corresponding to the names in the correct form
def set_x_and_y(x_name, y_name):
    pairs = df[[x_name, y_name]]
    x = np.log(pairs.iloc[:, 0])
    y = np.log(pairs.iloc[:, 1])
    return x , y

In [ ]:
def select_pair(stock_pairs):
    pair = randomizer(stock_pairs)
    x, y = set_x_and_y(pair[0], pair[1])
    return x, y

In [ ]:
def get_values(x, y):
    c, gamma, alpha, z = estimate_long_run_short_run_relationships(y,x)
    return c, gamma, alpha, z

In [ ]:
def get_order_book_for_instrument(instrument_id):
    """
    Fetching bid and ask books
    """
    
    book = e.get_last_price_book(instrument_id)
    
    return book.bids, book.asks

In [ ]:
# Step 1 - Best Bid/Ask for Pair 
# [TEST == True]
def bestBidAsk(x, y):
    """
    Returning current best bid/ask for pair
    """
    bids_x, asks_x = get_order_book_for_instrument(x.name)
    bids_y, asks_y = get_order_book_for_instrument(y.name)
    
    # Find "top of book"
    if len(bids_x) and len(asks_x) and len(bids_y) and len(asks_y):
        b_bid_x = bids_x[0]
        b_ask_x = asks_x[0]
        b_bid_y = bids_y[0]
        b_ask_y = asks_y[0]
        mid_x = (b_ask_x.price + b_bid_x.price)/2
        mid_y = (b_ask_y.price + b_bid_y.price)/2
    else:
        b_bid_x = None
        b_ask_x = None
        b_bid_y = None
        b_ask_y = None
        mid_x = None
        mid_y = None
    
    return b_bid_x, b_ask_x, mid_x, b_bid_y, b_ask_y, mid_y

In [ ]:
# Step 2 - Defining Expected Price for Y given X 
# [TEST == True]
def expected_price(b_ask_x):
    """
    Returns fair price given stock X"s price
    Applying Formula:  ln 𝑌𝑡 = 𝑐 + 𝛾 ln 𝑋𝑡 + 𝑧𝑡
    """
    b_bid_x, b_ask_x, mid_x, b_bid_y, b_ask_y, mid_y = bestBidAsk(x, y)
    price_y = c + gamma*np.log(b_ask_x.price)
    
    return price_y

In [ ]:
# Step 3 - Defining Volume Ratios from price of X and expected price of Y 
# [TEST == False, error from bid/ask not yet true]
def hedgeRatio(mid_x, price_y):
    """
    Returns pair volume ratio for hedging
    Applying Formula:  𝛾𝑌𝑡/𝑋𝑡
    """
    
    norm_price_y = np.exp(price_y) 
    hedge_ratio = gamma*(norm_price_y/mid_x)
    
    return hedge_ratio

In [ ]:
# [TEST == False, error from bid/ask not yet true]
def z_value(b_ask_x, b_ask_y):
    """
    Returns a normalised z-value (error term)
    Applying Formula: Z = Y_observed - Y_expected
    """
    observed = np.log(b_ask_y.price)
    expected = expected_price(b_ask_x)
    z = observed - expected
    
    return z

In [ ]:
# [TEST == True]
def delta_check(hedge_ratio, pos_x, pos_y):
    """
    Ensure positions are hedged before next iteration
    """
    # pos_x, pos_y = positions(x,y)
    delta = pos_x + pos_y/hedge_ratio
    
    if delta > -2 and delta < 2:
        return True

In [ ]:
def clear_all_positions():
    """
    Clear all positions regardless of losses incurred
    """
    print(e.get_positions())
    for s, p in e.get_positions().items():
        if p > 0:
            e.insert_order(s, price=1, volume=p, side="ask", order_type="ioc")
        elif p < 0:
            e.insert_order(s, price=100000, volume=-p, side="bid", order_type="ioc")  
    print(e.get_positions())

In [ ]:
# [TEST == True]
def positions(x, y):
    """
    Gather information on positions in each stock (x, y)
    """   
    pos_x = e.get_positions()[x.name]
    pos_y = e.get_positions()[y.name]
    
    return pos_x, pos_y 

In [ ]:
def printPositions():
    """
    For use in final 
    """
    positions = e.get_positions()
    for p in positions:
        print(p, positions[p])

In [ ]:
def sell(threshold, b_bid_x, b_ask_x, b_bid_y, b_ask_y):
    
    z = z_value(b_ask_x, b_ask_y)
    
    c_bid_x, c_ask_x, mid_x, c_bid_y, c_ask_y, mid_y = bestBidAsk(x, y)
    
    while c_bid_x is None:
        c_bid_x, c_ask_x, mid_x, c_bid_y, c_ask_y, mid_y = bestBidAsk(x, y)
    
    # Defining volume
    position_x, position_y  = positions(x, y)
    
    if position_x != 0 and position_y != 0:
        vol_y = abs(position_y)
        vol_x = abs(position_x)
    
        # If the z-score is positive, go long Y and go short X.
        if position_x < 0 and z > 0 and z < threshold/2:

            # Setting up orders (IOC)
            if vol_y == 0: # and c_ask_x.price < b_bid_x.price:
                buy_X = e.insert_order(x.name, price=c_ask_x.price, side="bid", volume=vol_x, order_type="ioc")
            elif vol_x == 0: # and c_bid_y.price > b_ask_y.price:
                sell_Y = e.insert_order(y.name, price=c_bid_y.price, side="ask", volume=vol_y, order_type="ioc")
            else: # c_bid_y.price > b_ask_y.price and c_ask_x.price < b_bid_x.price: 
                buy_X = e.insert_order(x.name, price=c_ask_x.price, side="bid", volume=vol_x, order_type="ioc")
                sell_Y = e.insert_order(y.name, price=c_bid_y.price, side="ask", volume=vol_y, order_type="ioc")
                print(f"Z > {threshold/2} (UNWINDING): Buy X @ {c_ask_x.price} and Sell Y @ {c_bid_y.price} with Volumes {vol_x, vol_y}")

        # If the z-score is negative, go long X and go short Y.
        if position_x > 0 and z < 0 and z > - threshold/2:

            # Setting up orders (IOC)
            if vol_y == 0: # and c_bid_x.price > b_ask_x.price:
                sell_X == e.insert_order(x.name, price=c_bid_x.price, side="ask", volume=vol_x, order_type="ioc")
            elif vol_x == 0: # and c_ask_y.price < b_bid_y.price:
                buy_Y = e.insert_order(y.name, price=c_ask_y.price, side="bid", volume=vol_y, order_type="ioc")
            else: #c_ask_y.price < b_bid_y.price and c_bid_x.price > b_ask_x.price:
                sell_X = e.insert_order(x.name, price=c_bid_x.price, side="ask", volume=vol_x, order_type="ioc")
                buy_Y = e.insert_order(y.name, price=c_ask_y.price, side="bid", volume=vol_y, order_type="ioc")
                print(f"Z < {threshold/2} (UNWINDING): Sell X @ {c_ask_x.price} and Buy Y @ {c_bid_y.price} with Volumes {vol_x, vol_y}")
    else:
        pass

In [ ]:
def strategy(threshold, x, y):
    """
    Strategy Description: 
     - If the z-value is positive, go long Y and go short X
     - If the z-score is negative, go long X and go short Y.
     
    Volume notes:
     - 
    """
    # Re-freshing best bid/ask
    # global b_bid_x, b_ask_x, b_bid_y, b_ask_y
    c, gamma, alpha, z = get_values(x, y)
    
    b_bid_x, b_ask_x, mid_x, b_bid_y, b_ask_y, mid_y = bestBidAsk(x, y)
    
    while b_bid_x is None:
        b_bid_x, b_ask_x, mid_x, b_bid_y, b_ask_y, mid_y = bestBidAsk(x, y)
        # return
    
    z = z_value(b_ask_x, b_ask_y)
    price_y = expected_price(b_ask_x)
    # Defining hedge ratio
    ratio = hedgeRatio(mid_x, price_y)
    
    # Gets positions in market to determine the volume in order to stay under the specified limits
    pos_x, pos_y = positions(x, y)
    
    # If the z-value is positive, go long Y and go short X. - ENTRY SIGNAL
    if z > threshold and delta_check(ratio, pos_x, pos_y):
        print("z>0.005")
        
        # Defining volume 
        vol_y = int(np.floor(min((b_bid_x.volume * ratio), b_ask_y.volume, 15)))
        vol_x = int(np.floor(vol_y / ratio))
        
        # Ensuring we are staying within the limits
        if pos_x - vol_x < -100:
            vol_x = abs(100 + pos_x)
            vol_y = abs(int(vol_x*ratio))

        if pos_y + vol_y > 100:
            vol_y = abs(100 - pos_y)
            vol_x = abs(int(vol_y/ratio))
                         
        # Setting up market orders (IOC)
        sell_X = e.insert_order(x.name, price=b_bid_x.price, side="ask", volume=vol_x, order_type="ioc")
        buy_Y = e.insert_order(y.name, price=b_ask_y.price, side="bid", volume=vol_y, order_type="ioc")

        # Ensuring fill of opposing trades
        trades_x = e.poll_new_trades(x.name)
        trades_y = e.poll_new_trades(y.name)

        if trades_x is None and trades_y is None:
            print("Trade Failed")

        while trades_x is None and trades_y.volume > 0:
            b_bid_x, b_ask_x, mid_x, b_bid_y, b_ask_y, mid_y = bestBidAsk(x, y)
            sell_X = e.insert_order(x.name, price=b_bid_x.price, side="ask", volume=vol_x, order_type="ioc")
            
        while trades_y is None and trades_x.volume > 0:
            b_bid_x, b_ask_x, mid_x, b_bid_y, b_ask_y, mid_y = bestBidAsk(x, y)
            buy_Y = e.insert_order(y.name, price=b_ask_y.price, side="bid", volume=vol_y, order_type="ioc")

        print(f"Z > 0: Sell X @ {b_bid_x.price} and Buy Y @ {b_ask_y.price} with Volumes {vol_x, vol_y}")
    
    # If the z-score is negative, go long X and go short Y.
    elif z < 0 - threshold and delta_check(ratio, pos_x, pos_y):
        print("z<0.005")
                         
        # Defining volume 
        vol_y = int(np.floor(min(b_ask_x.volume * ratio, b_bid_y.volume, 15)))
        vol_x = int(np.floor(vol_y / ratio))

        # Ensuring we are staying within the limits
        if pos_x - vol_x < -100:
            vol_x = 100 + pos_x
            vol_y = int(vol_x*ratio)

        if pos_y + vol_y > 100:
            vol_y = 100 - pos_y
            vol_x = int(vol_y/ratio)

        # Setting up market orders (IOC)
        sell_Y = e.insert_order(y.name, price=b_bid_y.price, side="ask", volume=vol_y, order_type="ioc")
        buy_X = e.insert_order(x.name, price=b_ask_x.price, side="bid", volume=vol_x, order_type="ioc")

        # Ensuring fill of opposing trades
        trades_x = e.poll_new_trades(x.name)
        trades_y = e.poll_new_trades(y.name)

        if trades_x is None and trades_y is None:
            print("Trade Failed")

        while trades_x is None and trades_y.volume > 0:
            b_bid_x, b_ask_x, mid_x, b_bid_y, b_ask_y, mid_y = bestBidAsk(x, y)
            buy_X = e.insert_order(x.name, price=b_ask_x.price, side="bid", volume=vol_x, order_type="ioc")

        while trades_y is None and trades_x.volume > 0:
            b_bid_x, b_ask_x, mid_x, b_bid_y, b_ask_y, mid_y = bestBidAsk(x, y)
            sell_Y = e.insert_order(y.name, price=b_bid_y.price, side="ask", volume=vol_y, order_type="ioc")

        print(f"Z < 0: Sell Y @ {b_bid_y.price} and Buy X @ {b_ask_x.price} with Volumes {vol_x, vol_y}")
        
    # sell(threshold)
    return b_bid_x, b_ask_x, b_bid_y, b_ask_y

In [ ]:
e = Exchange()
a = e.connect()
t = 0
executing = True
clear_all_positions()
threshold = 0.0025
stock_pairs = [('UNILEVER', 'TOTAL'),
 ('TOTAL', 'UNILEVER'),
 ('ASML', 'SAP'),
 ('SAP', 'ASML'),
 ('ALLIANZ', 'LVMH'),
 ('LVMH', 'ALLIANZ')]

while executing:
    clear_output(wait=True)
    print(f'')
    print(f'-----------------------------------------------------------------')
    print(f'TRADE LOOP ITERATION ENTERED AT {str(dt.datetime.now()):18s} UTC.')
    print(f'-----------------------------------------------------------------')
    print(f'')
    x, y = select_pair(stock_pairs)
    b_bid_x, b_ask_x, b_bid_y, b_ask_y = strategy(threshold, x, y)
    #time.sleep(1)
    sell(threshold, b_bid_x, b_ask_x, b_bid_y, b_ask_y)
    t += 1
    print(f"Cycle Count: {t} and Z-value: {z_value(b_ask_x, b_ask_y)}")
    printPositions()
    time.sleep(0.5)

In [ ]:
print(positions(x,y))

In [ ]:
b_bid_x, b_ask_x, mid_x, b_bid_y, b_ask_y, mid_y = bestBidAsk(x, y)
price_y = c + gamma*np.log(b_ask_x.price)
norm_price_y = np.exp(price_y) 
hedge_ratio = gamma*(mid_x/norm_price_y)
pos_x, pos_y = positions(x,y)
delta = pos_x + pos_y/hedge_ratio
print(f'{pos_x, pos_y}, {hedge_ratio}, {delta}')